In [185]:
import numpy as np
from numpy.random import randint


class Tensor(object):
    """Docstring for Tensor. """

    def __init__(self, data, label="", roots=None, how=None):
        # what would happen if you removed None?
        self.data = np.array(data)
        self.label = label
        self.roots = []
        
        # Each op/Tensor would have his own backward call.
        self.pass_the_grad = lambda : None 

        # Gradients >> Now everyone can get it...
        self.grad = np.ones(self.data.shape)
        

    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

    # Ops:
    def __add__(self, other):
        output = Tensor(self.data + other.data, label=self.label+"+"+other.label)
        output.roots = [self, other] 
        output.how = "+"
        
        def pass_the_gradient():
            """this function does:
            > Updates the gradients of the parents/roots.
            """
            if len(self.data.shape) == 1:
                self.grad = np.sum(output.grad, axis=0)
            else:
                self.grad = output.grad

            if len(self.data.shape) == 1:
                other.grad= np.sum(output.grad, axis=0)
            else:
                other.grad =  output.grad

        output.pass_the_grad = pass_the_gradient

        return  output 

    def __mul__(self, other):
        output = Tensor(self.data * other.data, label=self.label+"*"+other.label)
        output.roots = [self, other]
        output.how = "*"
        
        def pass_the_gradient():
            """this function does:
            > Updates the gradients of the parents/roots.
            """
            self.grad  = output.grad
            other.grad = output.grad

        output.pass_the_grad = pass_the_gradient
        return  output 
    
    def __matmul__(self, other):
        output = Tensor(self.data @ other.data, label=self.label+"@"+other.label)
        output.roots = [self, other]
        output.how = "@"

        def pass_the_gradient():
            """this function does:
            > Updates the gradients of the parents/roots.
            """
            self.grad  = output.grad @ other.data.T
            other.grad = self.data.T @ output.grad 
        output.pass_the_grad = pass_the_gradient
        
        return  output 

    def sigmoid(self):
        output = Tensor(1/(1+np.exp(-self.data)))
        output.roots = [self] 
        output.how = "σ"

        def pass_the_gradient():
            """this function does:
            > Updates the gradients of the parents/roots.
            """
            self.grad = output.data*(1-output.data)
        output.pass_the_grad = pass_the_gradient

        return output 
    
    def computation_g(self):
        # I love this function for real.
        roots  =  self.roots
        if len(roots)>0:
            ver, op = self.label, self.how
            yield (ver, op, [[r.label for r in roots]])
            for r in roots:
                yield from r.computation_graph()
                
    def computation_graph(self):
        v = self.computation_g()
        L = []
        for i in v:
            L.append(i)
        return L
        
        

X = Tensor(randint(10, size=(5,3)), label="X")
W1 = Tensor(randint(5, size=(3, 6)), label="W1")
b1 =  Tensor(np.ones((6,)), label="b1")

Z1 = X@W1
Z2 = Z1 + b1

L = Z2.sigmoid(); L.label = "L"

In [189]:
k = L.computation_graph()
k

[('L', 'σ', [['X@W1+b1']]),
 ('X@W1+b1', '+', [['X@W1', 'b1']]),
 ('X@W1', '@', [['X', 'W1']])]

In [191]:
for i in k:
    print(f"{i[0]} -> {i[1]}")

L -> σ
X@W1+b1 -> +
X@W1 -> @


In [ ]:
"""
print(f"X\n {X}")
print(f"W1\n {W1}")
print(f"b1\n {b1}")
print(f"Z1\n {Z1}")
print(f"Z2\n {Z2}")
print(f"L\n {L}")

l = [L, Z2, Z1, W1, b1]

print(type(L))
for i in l:
    print(f"Roots of {i.label}")
    print(f"{[r.label  for r in i.roots ]}")
"""

In [123]:
L.pass_the_grad()
Z2.pass_the_grad()
Z1.pass_the_grad()
W1.pass_the_grad()
b1.pass_the_grad()

Z1 = X@W1

In [23]:
X.grad, W1.grad

(array([[ 7., 15., 12.],
        [ 7., 15., 12.],
        [ 7., 15., 12.],
        [ 7., 15., 12.],
        [ 7., 15., 12.]]), array([[29., 29., 29., 29., 29., 29.],
        [19., 19., 19., 19., 19., 19.],
        [19., 19., 19., 19., 19., 19.]]))

In [13]:
X = Tensor(randint(4,size=(3,2)))
Y = Tensor(randint(4,size=(3,2)))

Z = X+Y

In [15]:
Z.pass_the_grad()

TypeError: pass_the_gradient() missing 2 required positional arguments: 'self' and 'other'

In [ ]:
!conda install -c conda-forge pygraphviz

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==2019.03=py37_0
  - defaults/linux-64::numba==0.43.1=py37h962f231_0
\ 

In [60]:
def g():
    yield("ff")
    yield("gg")

In [61]:
l = g()

In [62]:
type(l)

generator

In [64]:
print(l)

<generator object g at 0x7f4aac915e58>


In [ ]:
l